In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', 'src'))
sys.path.append(module_path)
import pandas as pd
import random
import scipy.signal as ss
import sys

import tools.data_reader_apd as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing

from scipy.fft import fft, fftfreq, fftshift
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


phases = {
    "Baseline": [dr.Phases.BASE_REST, dr.Phases.BASE_SPEECH],
    "Bug baseline": [dr.Phases.BUG_RELAX],
    "Speech baseline": [dr.Phases.SPEECH_RELAX],
    "Bug all": [dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE, dr.Phases.BUG_EXPOSURE, dr.Phases.BUG_BREAK, dr.Phases.BUG_REFLECT],
    "Speech all": [dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE, dr.Phases.SPEECH_EXPOSURE, dr.Phases.SPEECH_BREAK, dr.Phases.SPEECH_REFLECT],
    "Bug pre-anxiety": [dr.Phases.BUG_RELAX, dr.Phases.BUG_ANTICIPATE],
    "Speech pre-anxiety": [dr.Phases.SPEECH_RELAX, dr.Phases.SPEECH_ANTICIPATE],
    "Bug anxiety": [dr.Phases.BUG_EXPOSURE],
    "Speech anxiety": [dr.Phases.SPEECH_EXPOSURE],
    "Bug post-anxiety": [dr.Phases.BUG_BREAK, dr.Phases.BUG_REFLECT],
    "Speech post-anxiety": [dr.Phases.SPEECH_BREAK, dr.Phases.SPEECH_REFLECT],
}

test_phases = [
    phases["Baseline"],
    phases["Bug baseline"],
    phases["Speech baseline"],
    phases["Bug baseline"] + phases["Speech baseline"],
    phases["Baseline"] + phases["Bug baseline"],
    phases["Baseline"] + phases["Speech baseline"],
    phases["Baseline"] + phases["Bug baseline"] + phases["Speech baseline"],

    phases["Bug all"],
    phases["Speech all"],
    phases["Bug all"] + phases["Speech all"],
    phases["Baseline"] + phases["Bug all"],
    phases["Baseline"] + phases["Speech all"],
    phases["Baseline"] + phases["Bug all"] + phases["Speech all"],

    phases["Bug pre-anxiety"],
    phases["Speech pre-anxiety"],
    phases["Bug pre-anxiety"] + phases["Speech pre-anxiety"],
    phases["Baseline"] + phases["Bug pre-anxiety"],
    phases["Baseline"] + phases["Speech pre-anxiety"],
    phases["Baseline"] + phases["Bug pre-anxiety"] + phases["Speech pre-anxiety"],

    phases["Bug anxiety"],
    phases["Speech anxiety"],
    phases["Bug pre-anxiety"] + phases["Speech anxiety"],
    phases["Baseline"] + phases["Bug anxiety"],
    phases["Baseline"] + phases["Speech anxiety"],
    phases["Baseline"] + phases["Bug anxiety"] + phases["Speech anxiety"],

    phases["Bug post-anxiety"],
    phases["Speech post-anxiety"],
    phases["Bug post-anxiety"] + phases["Speech post-anxiety"],
    phases["Baseline"] + phases["Bug post-anxiety"],
    phases["Baseline"] + phases["Speech post-anxiety"],
    phases["Baseline"] + phases["Bug post-anxiety"] + phases["Speech post-anxiety"],

    phases["Bug pre-anxiety"] + phases["Bug anxiety"],
    phases["Speech pre-anxiety"] + phases["Speech anxiety"],
    phases["Bug pre-anxiety"] + phases["Bug anxiety"] + phases["Speech pre-anxiety"] + phases["Speech anxiety"],
    phases["Baseline"] + phases["Bug pre-anxiety"] + phases["Bug anxiety"],
    phases["Baseline"] + phases["Speech pre-anxiety"] + phases["Speech anxiety"],
    phases["Baseline"] + phases["Bug pre-anxiety"] + phases["Bug anxiety"] + phases["Speech pre-anxiety"] + phases["Speech anxiety"],

    phases["Bug post-anxiety"] + phases["Bug anxiety"],
    phases["Speech post-anxiety"] + phases["Speech anxiety"],
    phases["Bug post-anxiety"] + phases["Bug anxiety"] + phases["Speech post-anxiety"] + phases["Speech anxiety"],
    phases["Baseline"] + phases["Bug post-anxiety"] + phases["Bug anxiety"],
    phases["Baseline"] + phases["Speech post-anxiety"] + phases["Speech anxiety"],
    phases["Baseline"] + phases["Bug post-anxiety"] + phases["Bug anxiety"] + phases["Speech post-anxiety"] + phases["Speech anxiety"],

    phases["Bug pre-anxiety"] + phases["Bug post-anxiety"],
    phases["Speech pre-anxiety"] + phases["Speech post-anxiety"],
    phases["Bug pre-anxiety"] + phases["Bug post-anxiety"] + phases["Speech pre-anxiety"] + phases["Speech post-anxiety"],
    phases["Baseline"] + phases["Bug pre-anxiety"] + phases["Bug post-anxiety"],
    phases["Baseline"] + phases["Speech pre-anxiety"] + phases["Speech post-anxiety"],
    phases["Baseline"] + phases["Bug pre-anxiety"] + phases["Bug post-anxiety"] + phases["Speech pre-anxiety"] + phases["Speech post-anxiety"],
]